# Tensorflow
相比与Pytorch，Tensorflow更像是一门独立的语言，有着完整的生态。

In [1]:
# import库文件
from tensorflow.python.client import device_lib
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
# 打印硬件信息
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5595381858228685045
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16937379822857199316
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6837873514266135847
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
# 选择可用设备
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("All the available GPUs:\n", physical_devices)
if physical_devices:
    gpu = physical_devices[0]
    tf.config.experimental.set_memory_growth(gpu,True)
    tf.config.experimental.set_visible_devices(gpu, 'GPU')

All the available GPUs:
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
# 打印字符串
message = tf.constant("Welcome to the exciting world of Deep Neural Networks!")
with tf.compat.v1.Session() as sess:
    print(sess.run(message))

b'Welcome to the exciting world of Deep Neural Networks!'


### Tensorflow基础
Tensorflow的程序分为两部分：计算图的构建和生效。

> 计算图的构建：添加变量和操作，并按照逐层建立神经网络的顺序传递张量  
> 计算图的生效：使用tf.compat.v1.Session()定义一个会话对象，然后用run()方法运行。
其中run(fetches, feed_dict=None, options=None, run_metadata)，运算结果的值在fetches中提取

In [11]:
# 执行简单加法
tf.device('/device:XLA_GPU:0')
v_1 = tf.constant([1, 2, 3, 4]) # 添加变量
v_2 = tf.constant([2, 1, 5, 3])
v_add = tf.add(v_1, v_2) # v_add = v_1 + v_2，添加操作
with tf.compat.v1.Session() as sess:
    print(sess.run(v_add)) # 运行

[3 3 8 7]


Tensorflow的数据类型：
> tf.int32, tf.float32, tf.float64  
> tf.bool(tf.constant(\[True\]))  
> tf.string(tf.constant("Hello world."))  

Tensorflow支持的三种张量：
> 常量：其值不能改变的量  
> 变量：当一个量在会话中需要更新时使用。变量占用内存并需要初始化（通常表示权重和偏置），变量和初始化的分离，使所有关于图变量的赋值和计算都要通过tf.Session的run进行。初始化可以用sess.run(tf.global_variables_initializer())集体初始化，也可以单个初始化sess.run(x.initializer)  
> 占位符：待定变量，需要在调用时用feed_dict传入，无需初始化（tf.placeholder()在tensorflow 2.0被移除）  


In [11]:
# 标量常量/向量
t_1 = tf.constant(1) # 使用tf.constant(value, dtype)创建const Tensor
t_2 = tf.fill([2, 3], 2) # tf.fill(dimension, value)
t_3 = tf.zeros([2, 3], tf.int32) # t_3 = tf.constant([[0, 0, 0], [0, 0, 0]])
t_4 = tf.ones([2, 3], tf.int32) # tf.ones(dimension, dtype=float32)
list_np = np.arange(0, 5)
t = tf.convert_to_tensor(t_np, dtype=tf.int32) # 使用tf.convert_to_tensor(value, dtype)将numpy数据类型转化为tensor

# 变量，通过变量类创建
rand_t = tf.random.uniform([10, 10], 0, 10, seed=0) # 形状为[10, 10]，范围在[0, 10)的随机均匀分布
rand_s = tf.random.normal([10, 10], stddev=2) # 均值为0，标准差为2的正态分布
rand_st =  tf.random.truncated_normal([10, 10], mean=0, stddev=2) # 均值为0，标准差为2且所有数分布在2*sigma间
t_5 = tf.Variable(rand_t)
weights = tf.Variable(tf.random.normal([100, 100], stddev=2)) # 定义形状为[100, 100]，符合正态分布的权重矩阵
bias = tf.Variable(tf.zeros([100]), name='biases')
weight2 = tf.Variable(weights.initialized_value(), name='w2') # 用变量初始化

g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable("v", initializer=tf.zeros_initializer()(shape=[1])) # 图定义时的变量初始化

with tf.Session(graph=g1) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable("v")))

# 占位符 tf.placeholder(dype, shape=None, name=None)
x = tf.placeholder("float")
y = 2 * x
data = tf.random.uniform([4, 5], 10) # 定义一个方法
with tf.Session() as sess:
    x_data = sess.run(data) # 创建会话图并对需要提取的张量显示使用运行命令
    print(sess.run(y, feed_dict = {x: x_data}))

[0.]
[[15.656019   6.9269133 14.252337   2.2867928 10.612704 ]
 [ 5.1965218  5.3856497  5.419628  17.658213   3.7272491]
 [ 4.484955  15.107275   9.457092  13.465555   7.025157 ]
 [14.972277   8.535312   6.8968334  6.6343575 12.186405 ]]


tf.Variable(**initial_value=None**, trainable=True, collections=None, validate_shape=True, caching_device=None, **name=None**, variable_def=None, dtype=None, expected_shape=None, import_scope=None)  
tf.get_variable(**name**, shape=None, dtype=None, **initializer=None**, regularizer=None, trainable=True, collections=None, caching_device=None, partitioner=None, validate_shape=True, custom_getter=None):

> tf.Variable()在变量命名相同时自动处理为不同名字，而tf.get_variable()则会报错  
> 当tf.get_variable()的同名变量出现在同一变量域内，且reuse=True，则可以进行变量共享；
tf.get_variable()在tensorflow 2.0被移除

tf.variable_scope(scope)和tf.name_scope(scope):

> tf.variable_scope(scope)直接指定变量域，可以进行变量共享，与tf.get_variable()搭配使用。无论如何嵌套，若scope相同，直接跳转至第一次定义处，而且reuse=True无法变回False  
> tf.name_scope(scope)指定命名域，对变量的使用没有影响，主要是用来区分对象所处的神经网络的结构，并且能在TensorBoard中展示出来

In [14]:
tf.reset_default_graph()
with tf.variable_scope('conv1'):
    v1 = tf.get_variable('w1', initializer=1.0) # 注意：variable的默认类型为float32，直接赋1意味着类型为int32，会为后面的变量共享带来麻烦
with tf.variable_scope('conv1', reuse=True):
# 或写成tf.get_variable_scope().reuse_variables()，前者返回变量域，后者直接在变量域内修改reuse
    v2 = tf.get_variable('w1')
    x = v1 + v2
print(v1, v2)
print(x.op.name) # 算子所处变量域也会改变

<tf.Variable 'conv1/w1:0' shape=() dtype=float32_ref> <tf.Variable 'conv1/w1:0' shape=() dtype=float32_ref>
conv1_1/add


### Tensorflow的矩阵操作

In [15]:
sess = tf.InteractiveSession() # 创建会话图

I_matrix = tf.eye(5) # 定义一个5x5单位矩阵
print(I_matrix.eval()) # 单独print会显示该tensor的信息而非值

X = tf.Variable(tf.eye(10))
sess.run(X.initializer) # 变量初始化
A = tf.Variable(tf.random.normal([5, 10]))
sess.run(A.initializer)

product = tf.matmul(A, X) # 矩阵乘法
#print(product.eval())
b = tf.Variable(tf.random_uniform([5, 10], 0, 2, dtype=tf.int32))
sess.run(b.initializer)
#print(b.eval())
b_new = tf.cast(b, dtype=tf.float32) # 类型转换
t_sum = tf.add(product, b_new) # 矩阵加法
t_sub = product - b_new        # 矩阵减法
#print("A * X + b:\n", t_sum.eval())
#print("A * X - b:\n", t_sub.eval())

ele_wise_product = A * A # 对应元素相乘
scal_mul = tf.scalar_mul(2, A) # 标量乘法
ele_wise_div = tf.div(X, X) # 对应元素相除

min = tf.reduce_min(A) # 计算张量的最小值
max = tf.reduce_max(A) # 计算张量的最大值

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    #print(sess.run(ele_wise_product))

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


### TensorBoard可视化数据流图
TensorFlow 使用 TensorBoard 来提供计算图形的图形图像。这使得理解、调试和优化复杂的神经网络程序变得很方便。TensorBoard 也可以提供有关网络执行的量化指标。它读取 TensorFlow 事件文件，其中包含运行 TensorFlow 会话期间生成的摘要数据。

## 神经网络
### Batch_size与Epoch_size

> Batch_size:一次训练的样本数/进行一次调参所需的样本数  
> Epoch_size:将所有样本迭代的次数

区别：  
> 未使用Batch_size时，默认将所有数据输入网络并进行反向传播。这样做使得计算梯度的方向最准确，但是这样将导致总的参数调整次数变少（Epoch_size不变的情况下）  
> 随着Batch_size增大，方向越准确，但一次训练所需的内存增大，速度下降，一般随着Batch_size的增大需要适当增大Epoch_size来保证调参次数，同时学习率也要相应提高

对应的算法：
> BGD(Batch Gradient Decent，批量梯度下降法)：未使用Batch_size，梯度准确，非常耗时，可能因为神经网络是非凸的而收敛至局部最优点  
> SGD(Stochastic Gradient Descent，随机梯度下降法)：令Batch_size = 1，梯度方向不稳定，难以朝着正确方向前进，因此需要增大Epoch_size来保证学习率  
> mini-batch SGD：选择合适的Batch_size，缓解了GD算法直接掉入起始点附近的局部最优解的情况，同时由于梯度更准确，学习率需要相应提高

### 过拟合

在深度神经网络中存在过拟合的情况：

![过拟合](https://img-blog.csdnimg.cn/20190319100715848.PNG?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3h1YWhvMDkwNw==,size_16,color_FFFFFF,t_70)

在神经网络中，防止过拟合有以下方法：

#### 正则化

简单来说，正则化是一种为了减小测试误差的行为(有时候会增加训练误差)。我们在构造机器学习模型时，最终目的是让模型在面对新数据的时候，可以有很好的表现。当你用比较复杂的模型比如神经网络，去拟合数据时，很容易出现过拟合现象(训练集表现很好，测试集表现较差)，这会导致模型的泛化能力下降，这时候，我们就需要使用正则化，降低模型的复杂度。

模型的过拟合是因为考虑了过多的“不合适”样本点，这样，求导的时候，导数值很大，而自变量的值可大可小，所以意味着系数w要很大，所以当我们让w减少的时候，即意味着忽略这些样本点，即减小了模型的复杂度。

$L^p$范数的公式：$||x||_p = (\sum_{i} |x_i|^p)^{\frac{1}{p}}$。通过对所有数据额外加上该范数，完成正则化：
$\overline{J}(\omega, b) = J(\omega, b) + \frac{\lambda}{2m} L^p(\omega)$，其中$m$为样本个数而$\lambda$为控制正则化的超参数。

##### L1正则化

L1正则化通常用于进行特征选择。在求导后，得到$\omega_1 := \omega_1 - \alpha d\omega_1 = \omega_1 - \frac{\alpha\lambda}{2m}sign(\omega_1) - \frac{\partial J}{\partial \omega_1}$，若$\omega_1$为整数则减去一个固定的常数而若为负数则减去一个常数，最终使某些只被激活几次的神经元收敛至0。这样使得被小部分不适合的样本点影响的神经元能快速收敛至0，从而防止过拟合。

##### L2正则化

L2正则化通常用于进行模型简化。每次更新$\omega_1$时，需要$\omega_1 = (1 - \frac{\alpha\lambda}{m})\omega_1 - \frac{\partial J}{\partial\omega_1}$，每次更新时都减去一个数，使得权值趋向于0而不变为0（权值衰减，weight decay）

#### dropout

在每次迭代后随机删除一部分的结点，使得结点之间的关联性与复杂度减小。在大数据集中效果更佳。

#### earlystopping

提前终止，防止过拟合。